In [6]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare two ensemble outputs (e.g., gauge-based GMET and NLDAS-based GMET)
import matplotlib
matplotlib.use('Agg')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from   matplotlib.dates import DateFormatter
import os
import pandas as pd
import xarray as xr
import datetime

startTime = datetime.datetime.now()

def read_ens(out_forc_name_base, metric, start_yr, end_yr):
    for yr in range(start_yr, end_yr+1):        
        
        file = os.path.join(out_forc_name_base + '.' + str(yr) + '.'+metric+'.nc')
        f=xr.open_dataset(file)
        time = f['time'].values[:]
        pcp = f['pcp'].values[:]
        tmean = f['t_mean'].values[:]
        trange = f['t_range'].values[:]
        
        if yr == start_yr:
            time_concat = time
            pcp_concat = pcp
            tmean_concat = tmean
            trange_concat = trange
        else:
            time_concat = np.concatenate((time_concat,time), axis=0) # (time)
            pcp_concat = np.concatenate((pcp_concat, pcp), axis=0) # (time,y,x)
            tmean_concat = np.concatenate((tmean_concat, tmean), axis=0)
            trange_concat = np.concatenate((trange_concat, trange), axis=0)
            
    time_concat = pd.to_datetime(time_concat)        
        
    return time_concat, pcp_concat, tmean_concat, trange_concat

# ========================================================================================================================
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
stn_ens_dir = os.path.join(root_dir,'data/stn_ens_summary')
start_yr = 2016
end_yr = 2016

gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

result_dir = os.path.join(root_dir,'test_uniform_perturb')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
subforlder = 'gmet_ens_bc_summary'
file_basename = 'ens_forc'

time_format = '%Y-%m-%d'
plot_date_start = '2016-01-01'
plot_date_end = '2016-12-31'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)

formatter = DateFormatter('%Y/%m')
fmt_stn='b-o'
fmt='r-^'

line_marker_size = 0.75
line_width = 0.5
line_alpha = 0.8

scatter_marker_size = 3
scatter_alpha = 0.8

fc_stn = 'deepskyblue'
fc = 'salmon'

bound_alpha_stn = 0.6
bound_alpha = 0.6

dpi_value = 150

output_dir=os.path.join(root_dir,'scripts/step13_plot_temporal_stn_NLDAS')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# #======================================================================================================
# print('Read gridinfo mask')
# # get xy mask from gridinfo.nc
# f_gridinfo = xr.open_dataset(gridinfo_file)
# mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
# #data_mask = f_gridinfo['data_mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
# latitude = f_gridinfo['latitude'].values[:]
# longitude = f_gridinfo['longitude'].values[:]

# #======================================================================================================
# # read historical nldas data summary
# print('Read stn ens summary')

# for yr in range(start_yr, end_yr+1):
    
#     nldas_file = 'ens_forc.sumamry.'+str(yr)+'.nc'
#     nldas_path = os.path.join(stn_ens_dir, nldas_file)
    
#     f_stn = xr.open_dataset(nldas_path)
#     if yr == start_yr:
#         pcp_mean = f_stn['pcp_mean'].values[:] # (time, y, x). unit: mm/day
#         pcp_lb = f_stn['pcp_lb'].values[:]
#         pcp_ub = f_stn['pcp_ub'].values[:]
#         tmean_mean = f_stn['tmean_mean'].values[:] # unit: degC
#         tmean_lb = f_stn['tmean_lb'].values[:]
#         tmean_ub = f_stn['tmean_ub'].values[:]
#         trange_mean = f_stn['trange_mean'].values[:]
#         trange_lb = f_stn['trange_lb'].values[:]
#         trange_ub = f_stn['trange_ub'].values[:]
# #         time = pd.to_datetime(f_stn['time'].values[:]).strftime(time_format)
#         time = f_stn['time'].values[:]
#     else:
#         pcp_mean = np.concatenate((pcp_mean, f_stn['pcp_mean'].values[:]), axis = 0)
#         pcp_lb = np.concatenate((pcp_lb, f_stn['pcp_lb'].values[:]), axis = 0)
#         pcp_ub = np.concatenate((pcp_ub, f_stn['pcp_ub'].values[:]), axis = 0)
#         tmean_mean = np.concatenate((tmean_mean, f_stn['tmean_mean'].values[:]), axis = 0)
#         tmean_lb = np.concatenate((tmean_lb, f_stn['tmean_lb'].values[:]), axis = 0)
#         tmean_ub = np.concatenate((tmean_ub, f_stn['tmean_ub'].values[:]), axis = 0)
#         trange_mean = np.concatenate((trange_mean, f_stn['trange_mean'].values[:]), axis = 0)
#         trange_lb = np.concatenate((trange_lb, f_stn['trange_lb'].values[:]), axis = 0)
#         trange_ub = np.concatenate((trange_ub, f_stn['trange_ub'].values[:]), axis = 0)
# #         time = np.concatenate((time, pd.to_datetime(f_stn['time'].values[:]).strftime(time_format)), axis = 0)
#         time = np.concatenate((time, f_stn['time'].values[:]), axis = 0)

# # get time mask from nldas data
# # time_obj = np.asarray([datetime.datetime.strptime(t, time_format) for t in time])
# time_obj = pd.to_datetime(time)
# mask_t  = (time_obj >= plot_date_start_obj) & (time_obj <= plot_date_end_obj) 
# time_ens_stn = time_obj[mask_t]

# # convert masked values to nan
# pcp_mean=np.where(mask_xy==0,np.nan,pcp_mean)
# pcp_lb=np.where(mask_xy==0,np.nan,pcp_lb)
# pcp_ub=np.where(mask_xy==0,np.nan,pcp_ub)

# tmean_mean=np.where(mask_xy==0,np.nan,tmean_mean)
# tmean_lb=np.where(mask_xy==0,np.nan,tmean_lb)
# tmean_ub=np.where(mask_xy==0,np.nan,tmean_ub)

# trange_mean=np.where(mask_xy==0,np.nan,trange_mean)
# trange_lb=np.where(mask_xy==0,np.nan,trange_lb)
# trange_ub=np.where(mask_xy==0,np.nan,trange_ub)

# # caluclate xy mean (time)
# pcp_ensmean_stn = np.nanmean(np.nanmean(pcp_mean[mask_t,:,:], axis=2), axis=1)
# pcp_enslb_stn = np.nanmean(np.nanmean(pcp_lb[mask_t,:,:], axis=2), axis=1) 
# pcp_ensub_stn = np.nanmean(np.nanmean(pcp_ub[mask_t,:,:], axis=2), axis=1) 

# tmean_ensmean_stn = np.nanmean(np.nanmean(tmean_mean[mask_t,:,:], axis=2), axis=1) 
# tmean_enslb_stn = np.nanmean(np.nanmean(tmean_lb[mask_t,:,:], axis=2), axis=1)
# tmean_ensub_stn = np.nanmean(np.nanmean(tmean_ub[mask_t,:,:], axis=2), axis=1)

# trange_ensmean_stn = np.nanmean(np.nanmean(trange_mean[mask_t,:,:], axis=2), axis=1) 
# trange_enslb_stn = np.nanmean(np.nanmean(trange_lb[mask_t,:,:], axis=2), axis=1)
# trange_ensub_stn = np.nanmean(np.nanmean(trange_ub[mask_t,:,:], axis=2), axis=1)

# del pcp_mean,pcp_lb,pcp_ub,tmean_mean,tmean_lb,tmean_ub,trange_mean,trange_lb,trange_ub

#======================================================================================================
print('Plot')
# manual adjustment
vmin_prcp_mean = -1.0
vmax_prcp_mean = 20

vmin_tmean_mean = -12
vmax_tmean_mean = 35

vmin_trange_mean = 4
vmax_trange_mean = 20

# loop through all uniform tests
for test_folder in test_folders[-2:-1]:
    
    print(test_folder)
    test_dir = os.path.join(result_dir, test_folder)
    fig_title= test_folder

    # read ensemble mean    
    output_namebase = os.path.join(test_dir,subforlder, file_basename)
    metric = 'ensmean'
    time_ensmean, pcp_ensmean, tmean_ensmean, trange_ensmean = read_ens(output_namebase, metric, start_yr, end_yr)

    # read ensemble lower bound (5th percentile)  
    output_namebase = os.path.join(test_dir,subforlder, file_basename)
    metric = 'enspctl.5'
    time_enslb, pcp_enslb, tmean_enslb, trange_enslb = read_ens(output_namebase, metric, start_yr, end_yr)

    # read ensemble upper bound (95th percentile)  
    output_namebase = os.path.join(test_dir,subforlder, file_basename)
    metric = 'enspctl.95'
    time_ensub, pcp_ensub, tmean_ensub, trange_ensub = read_ens(output_namebase, metric, start_yr, end_yr)

     # convert masked values to nan
    pcp_ensmean=np.where(mask_xy==0,np.nan,pcp_ensmean)
    pcp_enslb=np.where(mask_xy==0,np.nan,pcp_enslb)
    pcp_ensub=np.where(mask_xy==0,np.nan,pcp_ensub)
    
    tmean_ensmean=np.where(mask_xy==0,np.nan,tmean_ensmean)
    tmean_enslb=np.where(mask_xy==0,np.nan,tmean_enslb)
    tmean_ensub=np.where(mask_xy==0,np.nan,tmean_ensub)
        
    trange_ensmean=np.where(mask_xy==0,np.nan,trange_ensmean)
    trange_enslb=np.where(mask_xy==0,np.nan,trange_enslb)
    trange_ensub=np.where(mask_xy==0,np.nan,trange_ensub)

    # define plot mask for nldas ensemble
    mask_ens_t = (time_ensmean>=plot_date_start_obj) & (time_ensmean<=plot_date_end_obj)
    time_ens = time_ensmean[mask_ens_t]
        
    # calculate mean and bounds
    pcp_ensmean = np.nanmean(np.nanmean(pcp_ensmean[mask_ens_t,:,:], axis=2),axis=1) # shape (time)
    pcp_enslb = np.nanmean(np.nanmean(pcp_enslb[mask_ens_t,:,:], axis=2),axis=1) # shape (time)
    pcp_ensub = np.nanmean(np.nanmean(pcp_ensub[mask_ens_t,:,:], axis=2),axis=1)

    tmean_ensmean = np.nanmean(np.nanmean(tmean_ensmean[mask_ens_t,:,:], axis=2),axis=1) 
    tmean_enslb = np.nanmean(np.nanmean(tmean_enslb[mask_ens_t,:,:], axis=2),axis=1) 
    tmean_ensub = np.nanmean(np.nanmean(tmean_ensub[mask_ens_t,:,:], axis=2),axis=1) 

    trange_ensmean = np.nanmean(np.nanmean(trange_ensmean[mask_ens_t,:,:], axis=2),axis=1) 
    trange_enslb = np.nanmean(np.nanmean(trange_enslb[mask_ens_t,:,:], axis=2),axis=1) 
    trange_ensub = np.nanmean(np.nanmean(trange_ensub[mask_ens_t,:,:], axis=2),axis=1) 
    
    # plot
    nrow=3 # # pcp, tmean, tmin, tmax, trange
    ncol=3 # time series, Q-Q plot
    fig = plt.figure(constrained_layout=False)
    fig.set_figwidth(3.54*ncol) 
    fig.set_figheight(3.54*0.75*nrow)
    fig.suptitle(fig_title, fontsize='medium', fontweight='semibold', color='g', y=1.0)
    
    mpl.rcParams['savefig.pad_inches'] = 0 # remove any padding from the edges of the figure when saved by savefig
    #     plt.annotate('Grid ('+str(y_id)+','+str(x_id)+')',(0.05,0.95),xycoords='figure fraction',fontsize='medium',fontweight='semibold')
    
    gs = fig.add_gridspec(nrow, ncol)
    ax00 = fig.add_subplot(gs[0, 0:2])
    ax01 = fig.add_subplot(gs[0, 2])
    ax10 = fig.add_subplot(gs[1, 0:2])
    ax11 = fig.add_subplot(gs[1, 2])
    ax20 = fig.add_subplot(gs[2, 0:2])
    ax21 = fig.add_subplot(gs[2, 2])
    
    # NLDAS vs. Ensemble [PCP]
    ax00.plot_date(time_ens_stn, pcp_ensmean_stn, fmt_stn, tz=None, linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='N15 ensemble mean') 
    ax00.plot_date(time_ens, pcp_ensmean, fmt, tz=None, linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='NLDAS ensemble mean')
    ax00.fill_between(time_ens_stn, pcp_enslb_stn, pcp_ensub_stn, linewidth=0, facecolor=fc_stn, alpha=bound_alpha_stn, label='N15 90% unc bounds')
    ax00.fill_between(time_ens, pcp_enslb, pcp_ensub, linewidth=0, facecolor=fc, alpha=bound_alpha, label='NLDAS 90% unc bounds')
    ax01.scatter(pcp_ensmean_stn, pcp_ensmean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)
    
    # NLDAS vs. Ensemble [TMEAN]
    ax10.plot_date(time_ens_stn, tmean_ensmean_stn, fmt_stn, tz=None, linewidth=line_width, markersize=line_marker_size, alpha=line_alpha) 
    ax10.plot_date(time_ens, tmean_ensmean, fmt, tz=None, linewidth=line_width, markersize=line_marker_size, alpha=line_alpha)
    ax10.fill_between(time_ens_stn, tmean_enslb_stn, tmean_ensub_stn, linewidth=0, facecolor=fc_stn, alpha=bound_alpha_stn)
    ax10.fill_between(time_ens, tmean_enslb, tmean_ensub, linewidth=0, facecolor=fc, alpha=bound_alpha)
    ax11.scatter(tmean_ensmean_stn, tmean_ensmean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)

    # NLDAS vs. Ensemble [TRANGE]
    ax20.plot_date(time_ens_stn, trange_ensmean_stn, fmt_stn, tz=None, linewidth=line_width, markersize=line_marker_size, alpha=line_alpha) 
    ax20.plot_date(time_ens, trange_ensmean, fmt, tz=None, linewidth=line_width, markersize=line_marker_size, alpha=line_alpha)
    ax20.fill_between(time_ens_stn, trange_enslb_stn, trange_ensub_stn, linewidth=0, facecolor=fc_stn, alpha=bound_alpha_stn)
    ax20.fill_between(time_ens, trange_enslb, trange_ensub, linewidth=0, facecolor=fc, alpha=bound_alpha)
    ax21.scatter(trange_ensmean_stn, trange_ensmean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)
        
    # 45 degree line in Q-Qplot
    axes = [ax01, ax11, ax21]
    for ax in axes:
        left, right = ax.get_xlim()
        bottom, top = ax.get_ylim()
        ax_min = min([left, bottom])
        ax_max = max([right, top])
        ax.set_xlim([ax_min, ax_max])
        ax.set_ylim([ax_min, ax_max])
        ax.plot([ax_min, ax_max],[ax_min, ax_max],color='grey',linewidth=1.0)
    
     # title
    ax00_title_str = '(a) Precip' 
    ax10_title_str = '(b) Tmean' 
    ax20_title_str = '(c) Trange'     
    axes_title_str=[ax00_title_str, ax10_title_str, ax20_title_str]
    axes=[ax00, ax10, ax20]
    for i in range(len(axes)):
        ax=axes[i]
        title_str=axes_title_str[i]
        ax.set_title(title_str, fontsize='small', fontweight='semibold')
                
    # x-axis label and legend
    axes=[ax00, ax10, ax20]
    for ax in axes:
        ax.set_xlabel('Date', fontsize='small')
        ax.set_xlim(left=plot_date_start_obj, right=plot_date_end_obj)
        ax.xaxis.set_major_formatter(formatter)
        ax.xaxis.set_tick_params(labelsize='small')#rotation=30,
        ax.legend(loc='upper right', fontsize='x-small', framealpha=0.5) 
    
    # y-axis label and limit
    for ax in [ax00]:
        ax.set_ylabel('Precipitation (mm/day)', fontsize='small')
        ax.set_ylim(bottom=vmin_prcp_mean, top=vmax_prcp_mean)
    for ax in [ax10]:
        ax.set_ylabel('Temperature ($^\circ$C)', fontsize='small')
        ax.set_ylim(bottom=vmin_tmean_mean, top=vmax_tmean_mean)
    for ax in [ax20]:
        ax.set_ylabel('Temperature ($^\circ$C)', fontsize='small')
        ax.set_ylim(bottom=vmin_trange_mean, top=vmax_trange_mean)
    
    for ax in [ax01, ax11, ax21]:
        ax.set_xlabel('N15 ensemble mean', fontsize='small')
        ax.set_ylabel('NLDAS ensemble mean', fontsize='small')
    
    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small') 
    
    fig.tight_layout()
    output_filename = test_folder+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)
    
    del time_ensmean, pcp_ensmean, tmean_ensmean, trange_ensmean
    del time_enslb, pcp_enslb, tmean_enslb, trange_enslb
    del time_ensub, pcp_ensub, tmean_ensub, trange_ensub

print('Done')


Plot
08953grids


No handles with labels found to put in legend.
No handles with labels found to put in legend.


Done


In [3]:
for test_folder in test_folders[-2:-1]:
    
    print(test_folder)


08953grids
